# Calculo de LTV y Cobertura Mensual 

![Guia de Valores Fasecolda](https://autosboutique.com/wp-content/uploads/2022/01/Fasecolda400-300x300.jpg)


*"Los valores de los vehículos de la Guía de valores de Fasecolda, son el resultado de un promedio de oferta de vehículo usado en el país según la oferta disponible en las diferentes fuentes del mercado clasificados, concesionarios e Internet."*


In [1]:
##CARGUE DE LIBRERIAS UTILES Y NECESARIAS
import pyodbc # Importando la librería pyodbc para conectarse a bases de datos ODBC
import pandas as pd # Importando la librería Pandas para trabajar con DataFrames
import numpy as np   # Importando la librería Numpy para operaciones matemáticas y estadísticas
import glob # Importando la librería Glob para obtener listas de archivos basadas en patrones
import re # Importando la librería re para trabajar con expresiones regulares
from alive_progress import alive_bar # Importando la barra de progreso alive_progress para mostrar  barras de progreso animadas en la consola
import datetime # Importando la librería datetime para trabajar con fechas y horas
import os # Importando la librería os para interactuar con el sistema operativo (paths, environment variables, etc.)
import timeit
import time
from datetime import datetime, timedelta, date
import calendar
pd.set_option('display.max_columns', None)

In [2]:
inicio = time.time()
time.sleep(1)

In [3]:
# Funciones

def first_day(d):
    if d.month == 1:
        return d.replace(month =12, year = d.year -1, day= 1)
    else:
        return d.replace(month= d.month -1, year= d.year, day = 1)
    
def last_day(d):
    if d.month == 1:
        return d.replace(month =12, year = d.year -1, day= 31)
    elif d.month == 3:
        if calendar.isleap(d.year) == True:
            return d.replace(month =d.month -1, year = d.year, day= 29)
        else:
            return d.replace(month =d.month -1, year = d.year, day= 28)
    elif (d.month == 5) | (d.month == 7) |(d.month == 10) | (d.month == 12):
        return d.replace(month =d.month -1, year = d.year, day= 30)
    else:
        return d.replace(month= d.month -1, year= d.year, day = 31)


**Se declaran la fecha correpondiente a los dias primero y ultimo del mes inmediatamente anterior**

In [4]:
# hoy = date.today()
# primero = first_day(hoy).strftime('%Y%m%d')
# ultimo = last_day(hoy).strftime('%Y%m%d')
# ayer =datetime.today()+timedelta(days=-1)
fecha_registro = 202501
ult_dia_mes = 20241231
primero = 20250130
ultimo = 20250131
ayer = '2025-05-02 14:42:16.563049'
ayer = datetime.strptime(ayer, '%Y-%m-%d %H:%M:%S.%f')
#año
a1 = 2025
print(primero, ultimo, ayer)

20250130 20250131 2025-05-02 14:42:16.563049


In [5]:
es_vehiculo = [150, 166, 319, 320, 348, 407, 428, 429, 442, 443, 453, 481, 482, 510, 512, 513, 521, 522, 523, 525, 537, 542 ]

**Se importa la base de datos de Fact_Garantias del periodo deseado (recordar que es mes vencido)**

In [6]:
cnxn = pyodbc.connect('DRIVER=SQL Server;Server=192.168.50.38\\DW_FZ;Database=DW_FZ;Port=1433;Trusted_Connection=yes')

Base de garantias 

In [7]:
query = f"""select * 
from (SELECT *
	  ,rank() over (partition by [vehiculo_marca],credito order by id_fecha desc) rank_id
     FROM [DW_FZ].[dbo].Fact_Garantias where id_fecha = {ultimo}) a
  where rank_id=1;
"""

garantias_total = pd.read_sql_query(query, cnxn)
garantias_total.info()

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_9536\2569993497.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  garantias_total = pd.read_sql_query(query, cnxn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 44 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id_fecha                   44898 non-null  int64         
 1   credito                    44898 non-null  float64       
 2   codigo_sucursal            44898 non-null  int64         
 3   numero_id_deudor           44898 non-null  float64       
 4   nombre_deudor              44898 non-null  object        
 5   direccion_domicilio        44898 non-null  object        
 6   ciudad                     44898 non-null  object        
 7   telefono_domicilio         44898 non-null  object        
 8   telefono_empresa           44898 non-null  object        
 9   telefono_celular           44898 non-null  object        
 10  valor_desembolso           44898 non-null  float64       
 11  fecha_desembolso           44898 non-null  datetime64[ns]
 12  fech

In [8]:
# query = f"""select * 
# from (SELECT *
# 	  ,rank() over (partition by [vehiculo_marca],credito order by id_fecha desc) rank_id
#      FROM [DW_FZ].[dbo].Fact_Garantias where id_fecha BETWEEN  {primero} AND {ultimo}) a
#   where rank_id=1;
# """

# garantias_total = pd.read_sql_query(query, cnxn)
# garantias_total.info()

In [9]:
garantias_total.tail()

,id_fecha,credito,codigo_sucursal,numero_id_deudor,nombre_deudor,direccion_domicilio,ciudad,telefono_domicilio,telefono_empresa,telefono_celular,valor_desembolso,fecha_desembolso,fecha_vencimiento,valor_saldo,dias_mora,valor_garantia,id_vehiculo,vehiculo_placa,vehiculo_marca,vehiculo_descripcion,vehiculo_color,vehiculo_modelo,vehiculo_codigo_fasecolda,vehiculo_cilindraje,vehiculo_numero_motor,vehiculo_numero_chasis,vehiculo_garantia_uso,codigo_garantia,concesionario,tipo_seguro_gps,gps_fecha_vencimiento,vlr_pri_seg_gps,numero_interno_poliza,numero_poliza,tipo_seguro_veh,ind_endo,poliza_fecha_inicio,poliza_fecha_vencimiento,valor_poliza,vlr_pri_seg_veh,memo,fecha_insercion,id_garantia,rank_id
44893,20250131,181878.0,114,3100375.0,MORENO CASTA#EDA JORGE,KRA 82 Ñ19 20,BOGOTA D.C.,000000091,,3102274677,131383973.0,2020-07-01,2025-07-01,21297306.0,29,227800000.0,31632,SZZ702,VOLQUETA Placa SZZ702,VOLQUETA Marca KENWORTH T460 Modelo 2013,BLANCO,2013,4426008,0,73367063,707661,PUB,1,IND.IVOR S.A. CASA I,0,1980-01-01,0.0,391259.0,,99,1,2020-07-01,2025-05-07,150000000.0,0.0,,2025-01-31 04:36:39.423,50852,1
44894,20250131,186413.0,114,11450013.0,MORENO CASTA#EDA ABDON,CRA 83 Ñ 24 A - 38,BOGOTA D.C.,004107907,,3132228485,600000000.0,2021-03-02,2027-03-02,350056085.0,28,204300000.0,45728,SZZ703,VOLQUETA Placa SZZ703,VOLQUETA Marca KENWORTH T460 Modelo 2013,BLANCO,2013,4422086,0,73367057,707660,PUB,1,IND.IVOR S.A. CASA I,0,1980-01-01,0.0,409353.0,62401,99,1,2020-11-30,2026-11-27,200700000.0,0.0,,2025-01-31 04:36:39.423,50851,1
44895,20250131,186413.0,114,11450013.0,MORENO CASTA#EDA ABDON,CRA 83 Ñ 24 A - 38,BOGOTA D.C.,004107907,,3132228485,600000000.0,2021-03-02,2027-03-02,350056085.0,28,227800000.0,45719,TAL640,VOLQUETA Placa TAL640,VOLQUETA Marca KENWORTH T460 Modelo 2013,BLANCO,2013,4426008,0,73360024,707650,PUB,1,IND.IVOR S.A. CASA I,0,1980-01-01,0.0,409355.0,62401,99,1,2020-11-30,2026-11-27,20700000.0,0.0,,2025-01-31 04:36:39.423,55177,1
44896,20250131,204394.0,114,3241030.0,CASALLAS CASALLAS EFREN,CL 14A Ñ 9 - 53 APTO 101,UBAQUE CUNDINAM,105720544,,3105720544,157000000.0,2022-02-02,2027-02-02,82793711.0,0,249700000.0,36594,TLZ310,VOLQUETA Placa TLZ310,VOLQUETA Marca HYUNDAI HD 320D MT 12900C,BLANCO,2014,03226010,0,D6CAD254489,KMFPA18SPEC078048,PUB,1,ALL CAR GROUP S.A.S,0,1980-01-01,0.0,603903.0,1300121,85,0,2023-04-29,2024-04-29,249700000.0,437989.0,,2025-01-31 04:36:39.423,157751,1
44897,20250131,202851.0,714,79109883.0,VARGAS MARIN CARLOS RAUL,CALLE 6A 61-51,GALAPA ATLANTIC,000000605,,3106999907,100000000.0,2021-12-29,2025-12-29,38084070.0,1,249900000.0,32417,TTY180,VOLQUETA Placa TTY180,VOLQUETA Marca KENWORTH T800 T800 MT TD,AMARILLO,2013,04426001,0,35305456,714328,PUB,1,DIRECTO,337,2025-12-29,68250.0,648391.0,0100323945,143,1,2023-10-14,2024-10-15,735900000.0,439522.0,,2025-01-31 04:36:39.423,156371,1


In [10]:
garantias_total = garantias_total[["credito", "vehiculo_codigo_fasecolda", "vehiculo_placa", "vehiculo_modelo", "valor_garantia"]]

In [11]:
garantias_total.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   credito                    44898 non-null  float64
 1   vehiculo_codigo_fasecolda  44898 non-null  object 
 2   vehiculo_placa             44898 non-null  object 
 3   vehiculo_modelo            44898 non-null  int64  
 4   valor_garantia             44898 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.7+ MB


Se importan los datos de la basa fasecolda

In [12]:
query = f"""
SELECT [Codigo], [Homologocodigo], [Marca], [Clase], [Referencia1], [Referencia2], [Referencia3], [Um],
[1970], [1971], [1972], [1973], [1974], [1975], [1976], [1977], [1978], [1979], [1980], [1981], [1982], 
[1983], [1984], [1985], [1986], [1987], [1988], [1989], [1990], [1991], [1992], [1993], [1994], [1995], 
[1996], [1997], [1998], [1999], [2000], [2001], [2002], [2003], [2004], [2005], [2006], [2007], [2008], 
[2009], [2010], [2011], [2012], [2013], [2014], [2015], [2016], [2017], [2018], [2019], [2020], [2021], 
[2022], [2023], [2024], [2025], [2026] 
FROM [Analitica].[dbo].[Guia_fasecolda_nuevo_formato];
 ;
""" 
GuiaFasecolda = pd.read_sql_query(query,cnxn)
GuiaFasecolda.info()

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_9536\1636760040.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  GuiaFasecolda = pd.read_sql_query(query,cnxn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17374 entries, 0 to 17373
Data columns (total 65 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Codigo          17374 non-null  object
 1   Homologocodigo  17374 non-null  object
 2   Marca           17374 non-null  object
 3   Clase           17374 non-null  object
 4   Referencia1     17374 non-null  object
 5   Referencia2     17374 non-null  object
 6   Referencia3     17374 non-null  object
 7   Um              17374 non-null  object
 8   1970            17374 non-null  object
 9   1971            17374 non-null  object
 10  1972            17374 non-null  object
 11  1973            17374 non-null  object
 12  1974            17374 non-null  object
 13  1975            17374 non-null  object
 14  1976            17374 non-null  object
 15  1977            17374 non-null  object
 16  1978            17374 non-null  object
 17  1979            17374 non-null  object
 18  1980  

merge de guia_fasecolda con garatias 

In [13]:
# Realizamos el merge entre los DataFrames garantias_total y GuiaFasecolda
df_merged_garantias_guia = pd.merge(garantias_total, GuiaFasecolda, left_on="vehiculo_codigo_fasecolda", right_on="Codigo", how="left")
df_merged_garantias_guia



,credito,vehiculo_codigo_fasecolda,vehiculo_placa,vehiculo_modelo,valor_garantia,Codigo,Homologocodigo,Marca,Clase,Referencia1,Referencia2,Referencia3,Um,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026
0,28860.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34858.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,42382.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,46238.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,54006.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,181878.0,4426008,SZZ702,2013,227800000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44894,186413.0,4422086,SZZ703,2013,204300000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44895,186413.0,4426008,TAL640,2013,227800000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44896,204394.0,03226010,TLZ310,2014,249700000.0,03226010,03265010,HYUNDAI,VOLQUETA,HD,320D,MT 12900CC TD 8X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,237200,249700,262800,0,0,0,0,0,0,0,0,0,0,295900


Se trae el historico de la Fact_Cartera_Mes, con el fin de obtener la garantia del primer comite 

In [14]:

query = """
WITH CTE AS (
    SELECT 
        credito, 
        valor_garantia,
        modelo_vehiculo,
        ROW_NUMBER() OVER (PARTITION BY credito ORDER BY id_fecha) AS row_num
    FROM Fact_Cartera_Mes
)
SELECT 
    credito, 
    valor_garantia,
    modelo_vehiculo
FROM CTE
WHERE row_num = 1
ORDER BY credito;
"""
hist_cartera = pd.read_sql(query, cnxn)
hist_cartera.info

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_9536\46807163.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  hist_cartera = pd.read_sql(query, cnxn)


<bound method DataFrame.info of          credito  valor_garantia  modelo_vehiculo
0         7285.0      12814000.0             1986
1        10449.0             0.0             1993
2        13548.0      11000000.0             1994
3        13745.0             0.0             1991
4        17160.0      28695000.0             1993
...          ...             ...              ...
200900  244520.0     175990000.0             2025
200901  244521.0      84100000.0             2021
200902  244522.0     104990000.0             2025
200903  398036.0      75000000.0             2005
200904  935994.0      53500000.0             2005

[200905 rows x 3 columns]>

Merge de guia_fasecolda & garantias con cartera_mes

In [15]:
df_merged = pd.merge(df_merged_garantias_guia, hist_cartera, left_on= "credito", right_on ="credito", how="left")
df_merged

,credito,vehiculo_codigo_fasecolda,vehiculo_placa,vehiculo_modelo,valor_garantia_x,Codigo,Homologocodigo,Marca,Clase,Referencia1,Referencia2,Referencia3,Um,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,valor_garantia_y,modelo_vehiculo
0,28860.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65000760.0,1993.0
1,34858.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25903000.0,2004.0
2,42382.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45990000.0,2007.0
3,46238.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13400000.0,1997.0
4,54006.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19500000.0,2000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,181878.0,4426008,SZZ702,2013,227800000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117000000.0,2013.0
44894,186413.0,4422086,SZZ703,2013,204300000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148500000.0,2013.0
44895,186413.0,4426008,TAL640,2013,227800000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148500000.0,2013.0
44896,204394.0,03226010,TLZ310,2014,249700000.0,03226010,03265010,HYUNDAI,VOLQUETA,HD,320D,MT 12900CC TD 8X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,237200,249700,262800,0,0,0,0,0,0,0,0,0,0,295900,177244900.0,2022.0


depreciación 

In [16]:
# Aseguramos que las columnas necesarias sean numéricas
df_merged["valor_garantia_y"] = pd.to_numeric(df_merged["valor_garantia_y"], errors='coerce')

columns_years = [str(year) for year in range(1970, 2027)]

# Crear una nueva columna 'valor_por_ano' para almacenar el valor de la columna correspondiente al año
df_merged["valor_por_ano"] = None

# Iterar sobre las filas del DataFrame combinado
for index, row in df_merged.iterrows():
    # Obtener el año de la columna 'vehiculo_modelo'
    year = str(row['vehiculo_modelo'])  # Convertir el año a string para la comparación
    
    # Verificar si el año está en las columnas de los años
    if year in columns_years:
        # Asignar el valor de la columna correspondiente al año
        df_merged.at[index, 'valor_por_ano'] = row[year]
    

# Aseguramos que la columna 'valor_por_ano' esté como numérica
df_merged['valor_por_ano'] = pd.to_numeric(df_merged['valor_por_ano'], errors='coerce')

# Multiplicamos 'valor_por_ano' por 1000
df_merged['valor_por_ano'] = df_merged['valor_por_ano'] * 1000



In [17]:
# En caso de que vehivulo modelo sea igual a  0 o vacio se toma el valro de modelo_vehiculo que viene de fact_cartera_mes
a1 = 2025  # año actual

# Crear la nueva columna 'valor_calculado' con la fórmula y redondear a 2 decimales
df_merged["valor_calculado"] = df_merged.apply(
    lambda row: round(min(0.05 * (a1 - (row['vehiculo_modelo'] if row['vehiculo_modelo'] not in [0, None, ''] else row['modelo_vehiculo'])), 1), 2) 
    if a1 > (row['vehiculo_modelo'] if row['vehiculo_modelo'] not in [0, None, ''] else row['modelo_vehiculo']) 
    else 0,
    axis=1
)

In [18]:
#prueba 
df_merged["valor_depreciado"] = df_merged.apply(
    lambda row: round(
        row['valor_garantia_y'] * (1 - row['valor_calculado']), 2
    ) if pd.isna(row['valor_por_ano'])  # Si valor_por_ano está vacío
    else round(
        row['valor_garantia_y'] * 0.75, 2  # Si valor_por_ano es 0, deprecia 25% de valor_garantia_y
    ) if row['valor_por_ano'] == 0  
    else round(
        row['valor_por_ano'] * 0.75, 2  # Si valor_por_ano tiene valor distinto de 0, deprecia 25% de valor_por_ano
    ), 
    axis=1
)



df_merged

,credito,vehiculo_codigo_fasecolda,vehiculo_placa,vehiculo_modelo,valor_garantia_x,Codigo,Homologocodigo,Marca,Clase,Referencia1,Referencia2,Referencia3,Um,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,valor_garantia_y,modelo_vehiculo,valor_por_ano,valor_calculado,valor_depreciado
0,28860.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65000760.0,1993.0,NaN,1.00,0.0
1,34858.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25903000.0,2004.0,NaN,1.00,0.0
2,42382.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45990000.0,2007.0,NaN,0.90,4599000.0
3,46238.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13400000.0,1997.0,NaN,1.00,0.0
4,54006.0,,,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19500000.0,2000.0,NaN,1.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44893,181878.0,4426008,SZZ702,2013,227800000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117000000.0,2013.0,NaN,0.60,46800000.0
44894,186413.0,4422086,SZZ703,2013,204300000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148500000.0,2013.0,NaN,0.60,59400000.0
44895,186413.0,4426008,TAL640,2013,227800000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148500000.0,2013.0,NaN,0.60,59400000.0
44896,204394.0,03226010,TLZ310,2014,249700000.0,03226010,03265010,HYUNDAI,VOLQUETA,HD,320D,MT 12900CC TD 8X4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,237200,249700,262800,0,0,0,0,0,0,0,0,0,0,295900,177244900.0,2022.0,249700000.0,0.55,187275000.0


In [637]:

# Imprimir el DataFrame con la nueva columna 'valor_calculado'
print(df_merged[["vehiculo_codigo_fasecolda", "vehiculo_modelo", "valor_calculado"]].head())


  vehiculo_codigo_fasecolda  vehiculo_modelo  valor_calculado
0                                          0             1.00
1                                          0             0.95
2                                          0             0.90
3                                          0             0.90
4                                          0             0.80


**Ahora se importan los datos de Cartera Mes**

#### cuando el codigo inicie en 888 y 999 no tienen garantias y lo que queda restante no se puede identificar.

## Carrocerias 

In [638]:
cyc = f"""Select t3.credito,
t3.id_fecha_desembolso,
LEFT(t3.id_fecha_desembolso, 6) AS anio
,t3.placa_vehiculo,
t4.precio_carroceria,
t4.precio_chasis,
t4.clasevehiculo,
t4.carroceria
from
(select t1.*,t2.numero_solicitud from
(select credito, placa_vehiculo, id_fecha_desembolso from dw_fz.dbo.Fact_Cartera_mes where id_fecha = 20250131 ) t1
left join (select credito, numero_solicitud from [DW_FZ].[dbo].[Fact_Cartera_Solicitud] group by credito,numero_solicitud)as t2 on t1.credito = t2.credito) t3
left join (select id_solicitud, precio_carroceria, precio_chasis, clasevehiculo, placa, carroceria
from [DW_FZ].[sol].[Dim_Sol_Vehiculos]) as t4 on t3.numero_solicitud = t4.id_solicitud and t3.placa_vehiculo = t4.placa
where t4.precio_carroceria >0 and ((t4.clasevehiculo != 'CAMIONETA PASAJ.' and (carroceria != 'VAN' or carroceria != 'FURGON')) or t4.clasevehiculo != 'AUTOMOVIL') order by t3.credito
"""
car_chas= pd.read_sql_query(cyc,cnxn)
car_chas.info()

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_2816\2173901832.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  car_chas= pd.read_sql_query(cyc,cnxn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7840 entries, 0 to 7839
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   credito              7840 non-null   float64
 1   id_fecha_desembolso  7840 non-null   object 
 2   anio                 7840 non-null   object 
 3   placa_vehiculo       7840 non-null   object 
 4   precio_carroceria    7840 non-null   int64  
 5   precio_chasis        7840 non-null   int64  
 6   clasevehiculo        7840 non-null   object 
 7   carroceria           7840 non-null   object 
dtypes: float64(1), int64(2), object(5)
memory usage: 490.1+ KB


In [639]:
# Convertimos la columna 'id_fecha_desembolso' en formato 'YYYYMMDD' a formato datetime
car_chas['id_fecha_desembolso'] = pd.to_datetime(car_chas['id_fecha_desembolso'], format='%Y%m%d')

# Fecha de hoy
hoy = datetime.today()

# Calculamos la diferencia en años
car_chas['anios_cumplidos'] = (hoy - car_chas['id_fecha_desembolso']).dt.days // 365


In [640]:
#depreciación de las carrocerías 

# Calcular la columna 'tiempo_depre' con la lógica del vehículo
car_chas['tiempo_depre'] = np.where(car_chas['clasevehiculo'] == 'BUS / BUSETA / MICROBUS', 20, 
                                     np.where(car_chas['carroceria'] == 'TERMO', 7, 10))

# Calcular 'vl_cria_depre' de manera eficiente
car_chas['vl_cria_depre'] = car_chas['precio_carroceria'] / car_chas['tiempo_depre']

# Calcular 'depreciado_hoy' como la multiplicación de 'anios_cumplidos' y 'vl_cria_depre'
car_chas['depreciado_hoy'] = car_chas['anios_cumplidos'] * car_chas['vl_cria_depre']

# Usar np.where para asignar el valor final de la carrocería
car_chas['valor_final_carroceria'] = np.where(car_chas['depreciado_hoy'] == 0, 
                                              car_chas['precio_carroceria'], 
                                              car_chas['precio_carroceria'] - car_chas['depreciado_hoy'])

# Convertir 'valor_final_carroceria' a entero si es necesario
car_chas['valor_final_carroceria'] = car_chas['valor_final_carroceria'].astype(int)
car_chas = car_chas[car_chas['precio_carroceria'] != 1]

# Ver las primeras filas del DataFrame para comprobar
car_chas.head()


,credito,id_fecha_desembolso,anio,placa_vehiculo,precio_carroceria,precio_chasis,clasevehiculo,carroceria,anios_cumplidos,tiempo_depre,vl_cria_depre,depreciado_hoy,valor_final_carroceria
0,168303.0,2019-06-28,201906,GFP841,33341000,140000000,CAMION,FURGON,5,10,3334100.0,16670500.0,16670500
1,168373.0,2019-06-28,201906,GET582,14200000,55990000,CAMION,ESTACAS,5,10,1420000.0,7100000.0,7100000
2,168425.0,2019-06-28,201906,GET645,35000000,46990000,CAMION,furgon,5,10,3500000.0,17500000.0,17500000
3,168539.0,2019-06-28,201906,GET545,23000000,51990000,CAMION,ESTACAS,5,10,2300000.0,11500000.0,11500000
4,169205.0,2019-07-22,201907,GET713,12500000,53899999,CAMION,ESTACAS,5,10,1250000.0,6250000.0,6250000


In [641]:
query = f"""
SELECT id_fecha, credito, id_fecha_desembolso, tipo_id_deudor, numero_id_deudor, id_vehiculo, placa_vehiculo, 
    marca_vehiculo, color_vehiculo, licencia_vehiculo, linea_vehiculo,tasa_inicial, tasa_actual, tasa_nominal,
    modelo_vehiculo, plazo_total, valor_desembolso, numero_dias_mora, codigo_oficial, 
    estado_cartera, valor_garantia, saldo_capital, total_deuda, tipo_producto 
FROM DW_FZ.dbo.Fact_Cartera_Mes
WHERE (id_fecha = N'{ultimo}') ;
""" 
cartera_mes = pd.read_sql_query(query,cnxn)
cartera_mes.info()

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_2816\4047010408.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cartera_mes = pd.read_sql_query(query,cnxn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42203 entries, 0 to 42202
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_fecha             42203 non-null  object 
 1   credito              42203 non-null  float64
 2   id_fecha_desembolso  42203 non-null  object 
 3   tipo_id_deudor       42203 non-null  int64  
 4   numero_id_deudor     42203 non-null  float64
 5   id_vehiculo          42203 non-null  int64  
 6   placa_vehiculo       42203 non-null  object 
 7   marca_vehiculo       42203 non-null  object 
 8   color_vehiculo       42203 non-null  object 
 9   licencia_vehiculo    42203 non-null  object 
 10  linea_vehiculo       42203 non-null  object 
 11  tasa_inicial         42203 non-null  float64
 12  tasa_actual          42203 non-null  float64
 13  tasa_nominal         42203 non-null  float64
 14  modelo_vehiculo      42203 non-null  int64  
 15  plazo_total          42203 non-null 

In [642]:
# Lectura de la base que indica el tipo de producto 
oficial_query = """select * from [Cartera175].[dbo].[Diccionario_Codigo_Oficial]"""

cod_oficial = pd.read_sql_query(oficial_query,cnxn)

cod_oficial.duplicated().sum()

cod_oficial = cod_oficial.drop_duplicates()

cod_oficial.head()

cod_ofi_lb = cod_oficial[cod_oficial['Categoria'] == 'Libre inversion']

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_2816\2276582978.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cod_oficial = pd.read_sql_query(oficial_query,cnxn)


**Ahora se importan los datos de Cartera Mes**

Identificacon tipo de credito y garantia 

In [643]:
# Creacion de base con los creditos que tienen garantia hipotecaria

#Lectura de la base que contiene los creditos con garantia hipotecaria
hipotecario = pd.read_excel(r'\\192.168.40.9\garaje$\BackUps_Analitica\BACKUP IVAN\ORGANIGRAMA\LTV\LTV MENSUAL\Hipotecas\HIPOTECAS.xlsx',skiprows=1)
hipotecario = hipotecario[['CREDITO','VALOR AVALÚO','FECHA AVALÚO']] # se toman solo las columnas que resultan utiles para el proceso
hipotecario.info()

hipotecario[['CREDITO','VALOR AVALÚO']].value_counts() #Se verifica los valores que contiene respecto a las varianñes credito y el valor del avaluo

hipotecario['año_avaluo'] = pd.to_datetime(hipotecario['FECHA AVALÚO'],format='mixed')
hipotecario['año_avaluo'] = [hipotecario['año_avaluo'][i].year for i in hipotecario.index]
hipotecario['año_avaluo'] = hipotecario['año_avaluo'].replace(1970,0)

print(hipotecario.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CREDITO       18 non-null     int64         
 1   VALOR AVALÚO  18 non-null     int64         
 2   FECHA AVALÚO  18 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(2)
memory usage: 564.0 bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CREDITO       18 non-null     int64         
 1   VALOR AVALÚO  18 non-null     int64         
 2   FECHA AVALÚO  18 non-null     datetime64[ns]
 3   año_avaluo    18 non-null     int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 708.0 bytes
None


In [644]:
año_actual = ayer.year


vlr_apreciacion = np.where(hipotecario['año_avaluo']!=0,(((año_actual - hipotecario['año_avaluo'])*0.04)+1),0)
apreciacion = hipotecario['VALOR AVALÚO'] * vlr_apreciacion

#A la data Hipotecario se adiciona la columna con el valor de apreciacion de cada inmueble
hipotecario['vlr_avaluo_apreciado'] = apreciacion.round(2)
hipotecario ['CREDITO'] = hipotecario['CREDITO'].astype(float)
print(hipotecario.info())
hipotecario

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   CREDITO               18 non-null     float64       
 1   VALOR AVALÚO          18 non-null     int64         
 2   FECHA AVALÚO          18 non-null     datetime64[ns]
 3   año_avaluo            18 non-null     int64         
 4   vlr_avaluo_apreciado  18 non-null     float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 852.0 bytes
None


,CREDITO,VALOR AVALÚO,FECHA AVALÚO,año_avaluo,vlr_avaluo_apreciado
0,214219.0,76000000,2016-11-09,2016,1.033600e+08
1,156806.0,3561000000,2018-10-03,2018,4.558080e+09
2,178584.0,112672000,2018-12-17,2018,1.442202e+08
3,169184.0,112800000,2019-05-31,2019,1.398720e+08
4,180396.0,187845000,2019-11-02,2019,2.329278e+08
5,183214.0,110382000,2020-08-06,2020,1.324584e+08
6,187180.0,1389765000,2020-11-12,2020,1.667718e+09
7,188717.0,148816821,2020-12-24,2020,1.785802e+08
8,217808.0,883284000,2019-01-08,2019,1.095272e+09
9,194613.0,1037680000,2019-01-08,2019,1.286723e+09


## Calculo valor garantia final 

In [645]:
## Combinacion de los df creados 
# Realizamos el primer merge entre 'hipotecario' y 'car_chas' basado en la columna 'credito'
df_combined = pd.merge(df_merged, car_chas, on='credito', how='left')

# Luego realizamos un segundo merge entre 'df_combined' y 'df_merged' basado en la columna 'credito'
df_combined2 = pd.merge(df_combined, hipotecario, left_on='credito', right_on='CREDITO' ,how='left')

df_combined2_unique = df_combined2.drop_duplicates(subset=['credito'], keep='first')

print(df_combined2_unique.info())

<class 'pandas.core.frame.DataFrame'>
Index: 42399 entries, 0 to 44897
Data columns (total 92 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   credito                    42399 non-null  float64       
 1   vehiculo_codigo_fasecolda  42399 non-null  object        
 2   vehiculo_placa             42399 non-null  object        
 3   vehiculo_modelo            42399 non-null  int64         
 4   valor_garantia_x           42399 non-null  float64       
 5   Codigo                     36796 non-null  object        
 6   Homologocodigo             36796 non-null  object        
 7   Marca                      36796 non-null  object        
 8   Clase                      36796 non-null  object        
 9   Referencia1                36796 non-null  object        
 10  Referencia2                36796 non-null  object        
 11  Referencia3                36796 non-null  object        
 12  Um       

In [646]:


df_final = pd.merge(cartera_mes, df_combined2_unique,  on='credito', how='left')

# Ver los primeros registros del DataFrame final
years_columns = [str(year) for year in range(1970, 2027)]

# Eliminar esas columnas del DataFrame
df_final = df_final.drop(columns=years_columns + ['CREDITO', 'Codigo', 'Homologocodigo', 'Marca', 'Clase', 'Referencia1', 'Referencia2', 'Referencia3', 'Um'])

# Verificar que las columnas se han eliminado
total_saldo_capital = df_final['saldo_capital'].sum()

# Imprimir el total
print(total_saldo_capital)

2058485230066.9


In [647]:
# # Verificar si hay duplicados en df_combined2
# duplicados_df_combined2 = df_combined2[df_combined2.duplicated(subset=['credito'], keep=False)]

# # Ver los duplicadosvalor_depreciadoFINA
# print(duplicados_df_combined2.info())

In [648]:
# Reemplazar valores NaN por 0 y realizar la suma
df_final['valor_garantia_final'] = (
    df_final['valor_depreciado'].fillna(0) +
    df_final['valor_final_carroceria'].fillna(0) + 
    df_final['vlr_avaluo_apreciado'].fillna(0)
)
#Observaciones
# Crear una nueva columna 'descripcion' que indique qué valores se tomaron para 'valor_garantia_final'
df_final['descripcion'] = np.where(
    (df_final['valor_depreciado'].notna()) & (df_final['valor_depreciado'] != 0) & (df_final['valor_final_carroceria'].notna()) & (df_final['vlr_avaluo_apreciado'].notna()),
    'valor_depreciado, valor_final_carroceria, vlr_avaluo_apreciado',  # Si las tres tienen valores
    np.where(
        (df_final['valor_depreciado'].notna()) & (df_final['valor_depreciado'] != 0) & (df_final['valor_final_carroceria'].notna()),
        'valor_depreciado, valor_final_carroceria',  # Si solo las dos primeras tienen valores
        np.where(
            (df_final['valor_depreciado'].notna()) & (df_final['valor_depreciado'] != 0) & (df_final['vlr_avaluo_apreciado'].notna()),
            'valor_depreciado, vlr_avaluo_apreciado',  # Si valor_depreciado y vlr_avaluo_apreciado tienen valores
            np.where(
                (df_final['valor_final_carroceria'].notna()) & (df_final['vlr_avaluo_apreciado'].notna()),
                'valor_final_carroceria, vlr_avaluo_apreciado',  # Si valor_final_carroceria y vlr_avaluo_apreciado tienen valores
                np.where(
                    df_final['valor_depreciado'].notna() & (df_final['valor_depreciado'] != 0),
                    'valor_depreciado',  # Si solo tiene valor_depreciado y no es 0
                    np.where(
                        df_final['valor_final_carroceria'].notna(),
                        'valor_final_carroceria',  # Si solo tiene valor_final_carroceria
                        np.where(
                            df_final['vlr_avaluo_apreciado'].notna(),
                            'vlr_avaluo_apreciado',  # Si solo tiene vlr_avaluo_apreciado
                            ''  # Si no tiene nada
                        )
                    )
                )
            )
        )
    )
)




In [649]:
# Crear una nueva columna 'tipo_de_garantia' con base en las condiciones
df_final['tipo_de_garantia'] = np.where(
    (df_final['valor_garantia_final'] == 0),
    'vehicular',  # Si valor_garantia_final es 0, asignar "vehicular"
    np.where(
        (df_final['vehiculo_codigo_fasecolda'].str.startswith('888')) | (df_final['vehiculo_codigo_fasecolda'].str.startswith('999')),
        'Otras',  # Si vehiculo_codigo_fasecolda empieza con 888 o 999, asignar "Otras"
        np.where(
            (df_final['valor_depreciado'].notna()) & (df_final['valor_final_carroceria'].notna()) & (df_final['vlr_avaluo_apreciado'].notna()),
            'vehicular/Hipotecaria',  # Si las tres tienen valores
            np.where(
                (df_final['valor_depreciado'].notna()) & (df_final['valor_final_carroceria'].notna()),
                'vehicular',  # Si valor_depreciado y valor_final_carroceria tienen valores
                np.where(
                    df_final['valor_depreciado'].notna(),
                    'vehicular',  # Si solo tiene valor_depreciado
                    np.where(
                        df_final['valor_final_carroceria'].notna(),
                        'Vehicular',  # Si solo tiene valor_final_carroceria
                        np.where(
                            df_final['vlr_avaluo_apreciado'].notna(),
                            'Hipotecaria',  # Si solo tiene vlr_avaluo_apreciado
                            ''  # Si no tiene nada
                        )
                    )
                )
            )
        )
    )
)



In [650]:

# LTV
# Realizamos el cálculo del LTV
df_final["LTV"] = np.where(
    df_final["saldo_capital"] <= 1, 
    np.nan, 
    df_final["saldo_capital"] / df_final["valor_garantia_final"]
)

# Redondeamos el valor de LTV a 9 decimales
# Redondeamos el valor de LTV a 9 decimales
df_final["LTV"] = df_final["LTV"].round(9)

# Reemplazar inf por NaN
df_final["LTV"].replace([np.inf, -np.inf], None, inplace=True)

# Cambiar el punto por la coma para visualización
df_final["LTV"] = df_final["LTV"].apply(lambda x: str(x).replace('.', ',') if pd.notna(x) else x)

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_2816\230293090.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final["LTV"].replace([np.inf, -np.inf], None, inplace=True)


In [651]:
# cobertura_garantia
# Realizamos el cálculo del LTV
df_final["cobertura_garantia"] = np.where(
    
    df_final["valor_garantia_final"] <= 1, 
    np.nan, 
    df_final["valor_garantia_final"] / df_final["saldo_capital"]
)

# Redondeamos el valor de LTV a 9 decimales
# Redondeamos el valor de LTV a 9 decimales
df_final["LTV"] = df_final["LTV"].round(9)

# Reemplazar inf por NaN
df_final["LTV"].replace([np.inf, -np.inf], None, inplace=True)

# Cambiar el punto por la coma para visualización
df_final["LTV"] = df_final["LTV"].apply(lambda x: str(x).replace('.', ',') if pd.notna(x) else x)



C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_2816\3665915747.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_final["LTV"].replace([np.inf, -np.inf], None, inplace=True)


In [652]:
df_final['codigo_oficial'] = df_final['codigo_oficial'].astype(int)
df_final_Cod = pd.merge(df_final, cod_oficial,  left_on="codigo_oficial", right_on="Codigo", how='left')

# Renombrar la columna 'Categoria' a 'libre_inversion_o_vehicular'
df_final_Cod['libre_inversion_o_vehicular'] = df_final_Cod['Categoria']

# Eliminar la columna original 'Categoria' si ya no la necesitas
df_final_Cod.drop(columns=['Categoria'], inplace=True)


df_final_Cod["es_vehiculo"] = np.where(df_final_Cod["libre_inversion_o_vehicular"] != 'Vehicular', "no", "si")

df_final_Cod



,id_fecha,credito,id_fecha_desembolso_x,tipo_id_deudor,numero_id_deudor,id_vehiculo,placa_vehiculo_x,marca_vehiculo,color_vehiculo,licencia_vehiculo,linea_vehiculo,tasa_inicial,tasa_actual,tasa_nominal,modelo_vehiculo_x,plazo_total,valor_desembolso,numero_dias_mora,codigo_oficial,estado_cartera,valor_garantia,saldo_capital,total_deuda,tipo_producto,vehiculo_codigo_fasecolda,vehiculo_placa,vehiculo_modelo,valor_garantia_x,valor_garantia_y,modelo_vehiculo_y,valor_por_ano,valor_calculado,valor_depreciado,id_fecha_desembolso_y,anio,placa_vehiculo_y,precio_carroceria,precio_chasis,clasevehiculo,carroceria,anios_cumplidos,tiempo_depre,vl_cria_depre,depreciado_hoy,valor_final_carroceria,VALOR AVALÚO,FECHA AVALÚO,año_avaluo,vlr_avaluo_apreciado,valor_garantia_final,descripcion,tipo_de_garantia,LTV,cobertura_garantia,Codigo,Producto,libre_inversion_o_vehicular,es_vehiculo
0,20250131,17160.0,19950818,1,7.938923e+07,31058,SFZ300,BUSETA,AZUL MARFI ROJO,PUB,G 30 5700 CC,58.0000,26.2999,1.96,1993,7,3696024.0,999,100,J,28695000.0,3.696024e+06,5.227274e+06,70,01606036,SFZ300,1993.0,28695000.0,28695000.0,1993.0,13000000.0,1.0,9750000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,9750000.0,valor_depreciado,vehicular,"0,379079385",2.637970,100,Conversion,Vehicular,si
1,20250131,20065.0,19970829,1,5.173294e+07,41,BFC080,,,PAR,KIA SEPHIA,42.4099,26.2999,1.96,2000,36,11200000.0,999,100,J,8700000.0,2.196019e+06,2.237519e+06,70,04601011,BFC080,2000.0,8700000.0,11200.0,2000.0,14900000.0,1.0,11175000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,11175000.0,valor_depreciado,vehicular,"0,196511736",5.088755,100,Conversion,Vehicular,si
2,20250131,20067.0,19970407,1,7.425572e+06,508,CRY637,,ROJO,PAR,FESTIVA CASUAL 1300,50.2300,26.2999,1.96,1997,48,10521000.0,999,100,J,10521000.0,9.037106e+06,1.433800e+07,70,03001001,CRY637,1997.0,10521000.0,10521000.0,1997.0,10200000.0,1.0,7650000.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,7650000.0,valor_depreciado,vehicular,"1,181321046",0.846510,100,Conversion,Vehicular,si
3,20250131,20112.0,19970623,1,7.923440e+07,11,AGE127,MAZDA,,PAR,323,46.7799,26.2999,1.96,0,36,2750000.0,999,100,J,2750001.0,1.966987e+06,3.603467e+06,70,05601030,AGE127,0.0,2750001.0,2750001.0,0.0,NaN,1.0,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,0.0,,vehicular,None,NaN,100,Conversion,Vehicular,si
4,20250131,20246.0,19980806,1,1.296672e+07,31335,SRD229,CHEVROLET,,PUB,,64.0000,26.2999,1.96,1995,36,10000000.0,999,100,J,33000000.0,4.368826e+06,8.243043e+06,70,,SRD229,1995.0,33000000.0,33000000.0,1995.0,NaN,1.0,0.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,0.0,,vehicular,None,NaN,100,Conversion,Vehicular,si
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42198,20250131,244518.0,20250131,1,2.820483e+07,84188,KXT683,VOLKSWAGEN,PLATA SIRIUS MET,PAR,GOL ^7! ^FL! COMFORT,24.8980,24.8980,1.86,2022,60,25000000.0,0,359,,55100000.0,2.500000e+07,2.500000e+07,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,0.0,,vehicular,None,NaN,359,Miguel Angel Romero Caceres,Vehicular,si
42199,20250131,244519.0,20250131,1,2.040515e+07,84196,NUY524,JAC,ROJO,PUB,HFC1120KN JRR EURO I,24.8980,24.8980,1.86,2025,72,148300000.0,0,310,,188990000.0,1.483000e+08,1.483000e+08,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,0.0,,vehicular,None,NaN,310,Robert Alejandro Banguera Nara,Vehicular,si
42200,20250131,244520.0,20250131,1,1.121833e+09,1,AVAL,FORD,BLANCO ARTICO,PUB,RANGER ^5! ^FL! XLS,24.8980,24.8980,1.86,2025,72,140792000.0,0,326,,175990000.0,1.407920e+08,1.407920e+08,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,0.0,,vehicular

In [653]:
query = f"""
SELECT 
    r.[credito],
    r.[tipo_id_deudor],
    r.[numero_id_deudor],
    r.[tipo_producto],
    r.[codigo_oficial],
    r.[saldo_capital],
    r.[id_fecha_desembolso],
    COUNT(DISTINCT g.[vehiculo_placa]) AS total_garantias
FROM 
    [DW_FZ].[dbo].[Fact_Cartera_Mes] r 
LEFT JOIN 
    [dbo].[Fact_Garantias] g 
ON 
    r.[credito] = g.[credito]
    AND g.[id_fecha] BETWEEN {primero}  AND {ultimo}  -- Asegurar que solo se consideran las fechas en el rango
WHERE 
    r.id_fecha = {ult_dia_mes}
    AND r.tipo_producto IN ('10','30','35','37','70','61','71')
    AND r.credito NOT IN ('176986','220804')
GROUP BY 
    r.[credito],
    r.[tipo_id_deudor],
    r.[numero_id_deudor],
    r.[tipo_producto],
    r.[codigo_oficial],
    r.[saldo_capital],
    r.[id_fecha_desembolso];
""" 
total_garantias = pd.read_sql_query(query,cnxn)
total_garantias.info()

C:\Users\cristian.parra\AppData\Local\Temp\ipykernel_2816\1657048087.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  total_garantias = pd.read_sql_query(query,cnxn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42188 entries, 0 to 42187
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   credito              42188 non-null  float64
 1   tipo_id_deudor       42188 non-null  int64  
 2   numero_id_deudor     42188 non-null  float64
 3   tipo_producto        42188 non-null  int64  
 4   codigo_oficial       42188 non-null  object 
 5   saldo_capital        42188 non-null  float64
 6   id_fecha_desembolso  42188 non-null  object 
 7   total_garantias      42188 non-null  int64  
dtypes: float64(3), int64(3), object(2)
memory usage: 2.6+ MB


In [654]:
df_final_LTV = pd.merge(df_final_Cod, total_garantias,  on="credito", how='left')

df_final_LTV['conteo_garantias'] = df_final_LTV['total_garantias']
df_final_LTV.drop(columns=[
    'tipo_id_deudor_y',
    'numero_id_deudor_y',
    'tipo_producto_y',
    'codigo_oficial_y',
    'saldo_capital_y',
    'id_fecha_desembolso',
    'total_garantias',
    'FECHA AVALÚO',
    'placa_vehiculo_y',
    'modelo_vehiculo_y',
    'valor_por_ano',
    'valor_calculado',
    'valor_depreciado',
    'id_fecha_desembolso_y',
    'precio_carroceria',
    'precio_chasis',
    'clasevehiculo',
    'carroceria',
    'anios_cumplidos',
    'tiempo_depre',
    'vl_cria_depre',
    'depreciado_hoy',
    'valor_final_carroceria',
    'VALOR AVALÚO',
    'año_avaluo',
    'vlr_avaluo_apreciado',
    'descripcion',
    'Codigo',
    'Producto',
    'total_deuda',
    'id_vehiculo',
    'anio',
    'valor_garantia_x',
    
], inplace=True)

df_final_LTV.rename(columns={
    'placa_vehiculo_x': 'placa_vehiculo',
    'id_fecha_desembolso_x': 'id_fecha_desembolso',
    'codigo_oficial_x': 'codigo_oficial', 
    'tipo_producto_x': 'tipo_producto',
    'tipo_id_deudor_x': 'tipo_id_deudor',
    'numero_id_deudor_x': 'numero_id_deudor',
    'saldo_capital_x': 'saldo_capital',
    'modelo_vehiculo_x':'modelo_vehiculo',
    'valor_garantia_y' :'vlr_gtia_a_tomar'
    
}, inplace=True)

# Colocar 'base_de_provision' como null (np.nan)
df_final_LTV['base_de_provision'] = np.nan

# Modificar la columna 'fecha' con los primeros 6 dígitos de 'ultimo'
# Asumiendo que 'ultimo' es una variable con valor '202501' (por ejemplo)

df_final_LTV['Fecha'] = str(fecha_registro)

# Crear la columna 'guia_fasecolda' con el valor especificado
df_final_LTV['Guia Fasecolda'] = '341_Enero_2025_Formato_Unico'

print(df_final_LTV.columns)


Index(['id_fecha', 'credito', 'id_fecha_desembolso', 'tipo_id_deudor',
       'numero_id_deudor', 'placa_vehiculo', 'marca_vehiculo',
       'color_vehiculo', 'licencia_vehiculo', 'linea_vehiculo', 'tasa_inicial',
       'tasa_actual', 'tasa_nominal', 'modelo_vehiculo', 'plazo_total',
       'valor_desembolso', 'numero_dias_mora', 'codigo_oficial',
       'estado_cartera', 'valor_garantia', 'saldo_capital', 'tipo_producto',
       'vehiculo_codigo_fasecolda', 'vehiculo_placa', 'vehiculo_modelo',
       'vlr_gtia_a_tomar', 'valor_garantia_final', 'tipo_de_garantia', 'LTV',
       'cobertura_garantia', 'libre_inversion_o_vehicular', 'es_vehiculo',
       'conteo_garantias', 'base_de_provision', 'Fecha', 'Guia Fasecolda'],
      dtype='object')


In [655]:
df_final_LTV = df_final_LTV.reindex(columns=['id_fecha', 'credito', 'id_fecha_desembolso', 'tipo_id_deudor',
       'numero_id_deudor', 'placa_vehiculo', 'marca_vehiculo',
       'linea_vehiculo', 'modelo_vehiculo', 'color_vehiculo',
       'licencia_vehiculo', 'tasa_inicial', 'tasa_actual', 'tasa_nominal',
       'valor_desembolso', 'numero_dias_mora', 'codigo_oficial',
       'estado_cartera', 'valor_garantia', 'saldo_capital', 'vlr_gtia_a_tomar',
       'base_de_provision', 'tipo_producto', 'valor_garantia_final',
       'conteo_garantias', 'es_vehiculo', 'LTV', 'cobertura_garantia',
       'Guia Fasecolda','Fecha','libre_inversion_o_vehicular', 'tipo_de_garantia',])
df_final_LTV

,id_fecha,credito,id_fecha_desembolso,tipo_id_deudor,numero_id_deudor,placa_vehiculo,marca_vehiculo,linea_vehiculo,modelo_vehiculo,color_vehiculo,licencia_vehiculo,tasa_inicial,tasa_actual,tasa_nominal,valor_desembolso,numero_dias_mora,codigo_oficial,estado_cartera,valor_garantia,saldo_capital,vlr_gtia_a_tomar,base_de_provision,tipo_producto,valor_garantia_final,conteo_garantias,es_vehiculo,LTV,cobertura_garantia,Guia Fasecolda,Fecha,libre_inversion_o_vehicular,tipo_de_garantia
0,20250131,17160.0,19950818,1,7.938923e+07,SFZ300,BUSETA,G 30 5700 CC,1993,AZUL MARFI ROJO,PUB,58.0000,26.2999,1.96,3696024.0,999,100,J,28695000.0,3.696024e+06,28695000.0,NaN,70,9750000.0,1.0,si,"0,379079385",2.637970,341_Enero_2025_Formato_Unico,202501,Vehicular,vehicular
1,20250131,20065.0,19970829,1,5.173294e+07,BFC080,,KIA SEPHIA,2000,,PAR,42.4099,26.2999,1.96,11200000.0,999,100,J,8700000.0,2.196019e+06,11200.0,NaN,70,11175000.0,1.0,si,"0,196511736",5.088755,341_Enero_2025_Formato_Unico,202501,Vehicular,vehicular
2,20250131,20067.0,19970407,1,7.425572e+06,CRY637,,FESTIVA CASUAL 1300,1997,ROJO,PAR,50.2300,26.2999,1.96,10521000.0,999,100,J,10521000.0,9.037106e+06,10521000.0,NaN,70,7650000.0,1.0,si,"1,181321046",0.846510,341_Enero_2025_Formato_Unico,202501,Vehicular,vehicular
3,20250131,20112.0,19970623,1,7.923440e+07,AGE127,MAZDA,323,0,,PAR,46.7799,26.2999,1.96,2750000.0,999,100,J,2750001.0,1.966987e+06,2750001.0,NaN,70,0.0,1.0,si,None,NaN,341_Enero_2025_Formato_Unico,202501,Vehicular,vehicular
4,20250131,20246.0,19980806,1,1.296672e+07,SRD229,CHEVROLET,,1995,,PUB,64.0000,26.2999,1.96,10000000.0,999,100,J,33000000.0,4.368826e+06,33000000.0,NaN,70,0.0,1.0,si,None,NaN,341_Enero_2025_Formato_Unico,202501,Vehicular,vehicular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42198,20250131,244518.0,20250131,1,2.820483e+07,KXT683,VOLKSWAGEN,GOL ^7! ^FL! COMFORT,2022,PLATA SIRIUS MET,PAR,24.8980,24.8980,1.86,25000000.0,0,359,,55100000.0,2.500000e+07,NaN,NaN,10,0.0,NaN,si,None,NaN,341_Enero_2025_Formato_Unico,202501,Vehicular,vehicular
42199,20250131,244519.0,20250131,1,2.040515e+07,NUY524,JAC,HFC1120KN JRR EURO I,2025,ROJO,PUB,24.8980,24.8980,1.86,148300000.0,0,310,,188990000.0,1.483000e+08,NaN,NaN,10,0.0,NaN,si,None,NaN,341_Enero_2025_Formato_Unico,202501,Vehicular,vehicular
42200,20250131,244520.0,20250131,1,1.121833e+09,AVAL,FORD,RANGER ^5! ^FL! XLS,2025,BLANCO ARTICO,PUB,24.8980,24.8980,1.86,140792000.0,0,326,,175990000.0,1.407920e+08,NaN,NaN,10,0.0,NaN,si,None,NaN,341_Enero_2025_Formato_Unico,202501,Vehicular,vehicular
42201,20250131,244521.0,20250131,1,7.973127e+07,FKN419,VOLKSWAGEN,T-CROSS COMFORTLINE,2021,GRIS PLATINO ME,PAR,23.7261,23.7261,1.78,70000000.0,0,359,,84100000.0,7.000000e+07,NaN,NaN,10,0.0,NaN,si,None,NaN,341_Enero_2025_Formato_Unico,202501,Vehicular,vehicular


In [656]:
df_final_LTV.to_excel(f"\\\\192.168.40.9\\garaje$\\BackUps_Analitica\\BACKUP IVAN\\ORGANIGRAMA\\LTV\\LTV MENSUAL\\OUTPUT LTV\\LTV_{ultimo}_nuevoformatoPruebaCifrasOf.xlsx", index=False)